<a href="https://colab.research.google.com/github/alexeynick/msu_oil_gas/blob/main/module2_lesson6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задачи модуля №2. Характеристики вытеснения

In [1]:
import pandas as pd
import numpy as np
import io

import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs import *
import matplotlib.pyplot as plt 

from sklearn.impute import KNNImputer

import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from plotly.subplots import make_subplots


In [2]:
anno_font = dict(
        family="arial bold",
        size=36,
        color="Black"
    )

In [3]:
df_object = pd.read_csv('https://raw.githubusercontent.com/alexeynick/msu_oil_gas/main/datasets/dynamic_object_5.csv')
df_object.tail(10)

,year,oil_year,production_frac,production_frac_rem,oil_total,production_frac_total,kin,water_year,water_total,water_cut,inject_year,inject_total,inject_comp_year,inject_comp_total
35,2005,55.9,0.6,2.4,6661,74.06,0.307,429.7,21933,87.0,410.6,22481.0,101.1,103.2
36,2006,57.7,0.6,2.5,6719,74.70,0.309,388.5,22322,85.2,361.6,22842.6,98.0,103.1
37,2007,56.7,0.6,2.6,6776,75.33,0.312,215.6,22538,73.7,117.5,22960.1,55.5,102.7
38,2008,61.1,0.7,2.8,6837,76.01,0.315,208.9,22747,70.7,156.2,23116.4,75.5,102.4
39,2009,62.1,0.7,3.0,6899,76.70,0.318,202.4,22949,69.3,137.2,23253.5,68.2,102.1
40,2010,60.3,0.7,3.0,6959,77.37,0.320,205.3,23154,70.6,141.2,23394.8,69.5,101.8
41,2011,51.5,0.6,2.6,7011,77.94,0.323,192.5,23347,73.2,138.9,23533.6,73.4,101.6
42,2012,49.6,0.6,2.6,7061,78.49,0.325,201.8,23548,75.4,144.5,23678.1,73.3,101.4
43,2013,44.6,0.5,2.4,7105,78.99,0.327,213.5,23762,79.1,146.7,23824.9,71.1,101.1
44,2014,47.3,0.5,2.6,7153,79.52,0.329,231.3,23993,79.5,107.7,23932.6,48.2,100.6


In [4]:
df_graph = df_object[['year', 'oil_total', 'water_total']].copy()
df_graph['oil_total'] = df_graph['oil_total'].astype(float)
df_graph['water_total'] = df_graph['water_total'].astype(float)
df_graph['water_total_deg_1'] = np.power(df_graph['water_total'].astype(float), -1)
df_graph['water_total_ln'] = np.log(df_graph['water_total'].astype(float))
df_graph['oil_total_ln'] = np.log(df_graph['oil_total'].astype(float))
df_graph['water_to_oil_total'] = df_graph['water_total'].astype(float) / df_graph['oil_total'].astype(float)
df_graph.head(5)

,year,oil_total,water_total,water_total_deg_1,water_total_ln,oil_total_ln,water_to_oil_total
0,1970,90.0,92.0,0.010870,4.521789,4.499810,1.022222
1,1971,181.0,190.0,0.005263,5.247024,5.198497,1.049724
2,1972,359.0,378.0,0.002646,5.934894,5.883322,1.052925
3,1973,754.0,796.0,0.001256,6.679599,6.625392,1.055703
4,1974,1175.0,1324.0,0.000755,7.188413,7.069023,1.126809


In [5]:
niz_total = 8995

In [6]:
def kambarov_func(x, a, b):
    return a - b * (x ** -1)

def kambarov_niz(a, b):
    return a - (b * 0.02) ** 0.5

def sazonov_func(x, a, b):
    return a + b * np.log(x)

def sazonov_niz(a, b):
    return a + b * np.log(b / 0.02)

def french_func(x, a, b):
    return (a + 1) * x + b * (x ** 2)

def french_niz(a, b):
    fvk = 0.98
    return (1 / (2 * b))  * (fvk / (1 - fvk) - a)

def naz_sip_func(x, a, b):
    return (a * x - b * (x ** 2)) / (1 - b * x)

def naz_sip_niz(vvk, a, b):
    return vvk / (a - 1 + b * vvk)

In [7]:
xdata = df_graph[-25:]['water_total']
ydata = df_graph[-25:]['oil_total']

popt, pcov = curve_fit(kambarov_func, xdata, ydata)
popt

#print(popt)
#print(kambarov_niz(*popt))

#ydata_fit = func_kambarov(xdata, *popt)

array([9.18132561e+03, 5.33728279e+07])

In [8]:
fig = make_subplots(
    subplot_titles=[
                    'Весь период',
                    'Период аппроксимации'],
    rows=1, 
    cols=2)

fig.add_trace(go.Scatter(x=df_graph['water_total_deg_1'], y=df_graph['oil_total'],
                    mode='lines+markers',
                    name='Накопленный объем нефти, 10^6'),
    row=1, col=1)

fig.add_trace(go.Scatter(x=df_graph[-35:]['water_total_deg_1'], y=df_graph[-35:]['oil_total'],
                    mode='lines+markers',
                    name='Период аппроксимации'),
    row=1, col=1)

#####

fig.add_trace(go.Scatter(x=df_graph[-35:]['water_total_deg_1'], y=df_graph[-35:]['oil_total'],
                    mode='lines+markers',
                    name='Годовой отбор жидкости, 10^6'),
    row=1, col=2)

fig.add_trace(go.Scatter(x=df_graph[-25:]['water_total_deg_1'], y=df_graph[-25:]['oil_total'],
                    mode='lines+markers',
                    name='Участок аппроксимации'),
    row=1, col=2)

fig.add_trace(go.Scatter(x=df_graph[-25:]['water_total_deg_1'], y=kambarov_func(xdata, *popt),
                    mode='lines',
                    name='Функция аппроксимации (Камбарова)'),
    row=1, col=2)


fig['layout']['xaxis1']['title']='1 / Накопленный объем жидкости, 10^-6'
fig['layout']['xaxis1']['tickangle']=-45
fig['layout']['xaxis1']['tickmode']='array'
fig['layout']['yaxis1']['title']='Накопленный объем нефти, 10^6'

fig['layout']['xaxis2']['title']='1 / Накопленный объем жидкости, 10^-6'
fig['layout']['xaxis2']['tickangle']=-45



fig.update_layout( 
    title={
        'text': 'Зависимость накопленной добычи нефти от объема жидкости',
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'},
    autosize=False,
    width=1400,
    height=700)


fig.show()

In [9]:
frame_range = [10, 25]
shift_range = [0, 15]

In [10]:
data = []

for shift in range(shift_range[0], shift_range[1] + 1):
  for frame in range(frame_range[0], frame_range[1] + 1):
    if shift == 0:
      xdata = df_graph[-frame:]['water_total']
      ydata = df_graph[-frame:]['oil_total']
    else:
      xdata = df_graph[-(frame + shift):-shift]['water_total']
      ydata = df_graph[-(frame + shift):-shift]['oil_total']      

    popt, pcov = curve_fit(kambarov_func, xdata, ydata)
    popt

    data.append([shift, frame, kambarov_niz(*popt)])

    #print(popt[0], popt[1], np.abs(kambarov_niz(popt[0], popt[1]) - niz_total))

df_approx = pd.DataFrame(data, columns=['shift', 'frame', 'niz'])
df_approx['err'] = np.abs(df_approx['niz'] - niz_total)

In [11]:
df_approx.head(5)

,shift,frame,niz,err
0,0,10,11114.592887,2119.592887
1,0,11,10672.699226,1677.699226
2,0,12,10302.446703,1307.446703
3,0,13,9960.872802,965.872802
4,0,14,9677.561126,682.561126


In [12]:
def print_approx_graph(df_approx):
  fig = go.Figure()

  for shift in range(shift_range[0], shift_range[1] + 1):
    df_shift_data = df_approx.query('shift == @shift')

    fig.add_trace(go.Scatter(x=df_shift_data['frame'], y=df_shift_data['err'],
                        mode='lines',
                        name='График при сдвиге: {0}'.format(shift)))

  fig.update_layout( 
      xaxis=dict(
          domain=[0.2, 0.9],
          title='Размер окна',
      ),
      yaxis=dict(
          title='Ошибка предсказания',
      ),
      title={
          'text': "Зависимость ошибки предсказания НИЗ от параметров аппроксимации",
          'y':0.95,
          'x':0.50,
          'xanchor': 'center',
          'yanchor': 'top'},
      autosize=False,
      width=1400,
      height=700
  )

  fig.show()

In [13]:
print_approx_graph(df_approx)

In [15]:
def df_to_plotly(df):
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}

In [45]:
def plot_approx_heatmap(df_approx_pvt):
  fig = go.Figure(data=go.Heatmap(
          df_to_plotly(df_approx_pvt),
          colorscale='blues'))

  fig.update_layout( 
      xaxis=dict(
          title='Размер окна',
      ),
      yaxis=dict(
          title='Сдвиг',
      ),
      title={
          'text': "Зависимость ошибки предсказания НИЗ от параметров аппроксимации",
          'y':0.95,
          'x':0.50,
          'xanchor': 'center',
          'yanchor': 'top'},
      autosize=False,
      width=1400,
      height=700
  )

  fig.show()  

In [46]:
df_approx_pvt = df_approx.pivot(index='shift', columns='frame', values='err')

plot_approx_heatmap(df_approx_pvt)

In [17]:
xdata = df_graph[-17:-1]['water_total']
ydata = df_graph[-17:-1]['oil_total']

popt, pcov = curve_fit(kambarov_func, xdata, ydata)

kambarov_niz(*popt)

8993.317215475843

In [18]:
fig = make_subplots(
    subplot_titles=[
                    'Весь период',
                    'Период аппроксимации'],
    rows=1, 
    cols=2)

fig.add_trace(go.Scatter(x=df_graph['water_total_deg_1'], y=df_graph['oil_total'],
                    mode='lines+markers',
                    name='Накопленный объем нефти, 10^6'),
    row=1, col=1)

fig.add_trace(go.Scatter(x=df_graph[-35:]['water_total_deg_1'], y=df_graph[-35:]['oil_total'],
                    mode='lines+markers',
                    name='Период аппроксимации'),
    row=1, col=1)

#####

fig.add_trace(go.Scatter(x=df_graph[-35:]['water_total_deg_1'], y=df_graph[-35:]['oil_total'],
                    mode='lines+markers',
                    name='Годовой отбор жидкости, 10^6'),
    row=1, col=2)

fig.add_trace(go.Scatter(x=df_graph[-17:-1]['water_total_deg_1'], y=df_graph[-17:-1]['oil_total'],
                    mode='lines+markers',
                    name='Участок аппроксимации'),
    row=1, col=2)

fig.add_trace(go.Scatter(x=df_graph[-17:-1]['water_total_deg_1'], y=kambarov_func(xdata, *popt),
                    mode='lines',
                    name='Функция аппроксимации (Камбарова)'),
    row=1, col=2)

fig.add_annotation(x=10**(-4), y=7000, text="НИЗ={0}".format(int(kambarov_niz(*popt))), showarrow=False, yshift=10, font=anno_font,
    row=1, col=2)

fig['layout']['xaxis1']['title']='1 / Накопленный объем жидкости, 10^-6'
fig['layout']['xaxis1']['tickangle']=-45
fig['layout']['xaxis1']['tickmode']='array'
fig['layout']['yaxis1']['title']='Накопленный объем нефти, 10^6'

fig['layout']['xaxis2']['title']='1 / Накопленный объем жидкости, 10^-6'
fig['layout']['xaxis2']['tickangle']=-45



fig.update_layout( 
    title={
        'text': 'Зависимость накопленной добычи нефти от объема жидкости',
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'},
    autosize=False,
    width=1400,
    height=700)


fig.show()

In [19]:
xdata = df_graph[-20:]['oil_total']
ydata = df_graph[-20:]['water_total']

popt, pcov = curve_fit(french_func, xdata, ydata)
popt

#print(popt)
#print(kambarov_niz(*popt))

ydata_fit = french_func(xdata, *popt)
ydata_fit

25    18242.958836
26    18516.481317
27    18803.318116
28    19126.495760
29    19429.452170
30    19734.777102
31    20059.957993
32    20417.181816
33    20736.030848
34    21081.237008
35    21411.207890
36    21761.940318
37    22109.389191
38    22484.255083
39    22868.481401
40    23243.399398
41    23570.783699
42    23887.726430
43    24168.379751
44    24476.408775
Name: oil_total, dtype: float64

In [20]:
french_niz(*popt)

58816.89903729069

In [21]:
popt

array([-5.93831446e-01,  4.21595088e-04])

In [22]:
fig = make_subplots(
    subplot_titles=[
                    'Весь период',
                    'Период аппроксимации'],
    rows=1, 
    cols=2)

fig.add_trace(go.Scatter(x=df_graph['oil_total'], y=df_graph['water_to_oil_total'] - 1,
                    mode='lines+markers',
                    name='Накопленный объем нефти, 10^6'),
    row=1, col=1)

fig.add_trace(go.Scatter(x=df_graph[-35:]['oil_total'], y=df_graph[-35:]['water_to_oil_total'] - 1,
                    mode='lines+markers',
                    name='Период аппроксимации'),
    row=1, col=1)

#####

fig.add_trace(go.Scatter(x=df_graph[-35:]['oil_total'], y=df_graph[-35:]['water_to_oil_total'] - 1,
                    mode='lines+markers',
                    name='Годовой отбор жидкости, 10^6'),
    row=1, col=2)

fig.add_trace(go.Scatter(x=df_graph[-25:]['oil_total'], y=df_graph[-25:]['water_to_oil_total'] - 1,
                    mode='lines+markers',
                    name='Участок аппроксимации'),
    row=1, col=2)

fig.add_trace(go.Scatter(x=df_graph[-20:]['oil_total'], y= (french_func(xdata, *popt) / df_graph[-20:]['oil_total']) - 1,
                    mode='lines',
                    name='Функция аппроксимации (Камбарова)'),
    row=1, col=2)


fig['layout']['xaxis1']['title']='1 / Накопленный объем нефти, 10^-6'
fig['layout']['xaxis1']['tickangle']=-45
fig['layout']['xaxis1']['tickmode']='array'
fig['layout']['yaxis1']['title']='Соотношение вода/нефти, 10^6'

fig['layout']['xaxis2']['title']='1 / Накопленный объем нефти, 10^-6'
fig['layout']['xaxis2']['tickangle']=-45



fig.update_layout( 
    title={
        'text': 'Зависимость накопленной добычи нефти от объема жидкости',
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'},
    autosize=False,
    width=1400,
    height=700)


fig.show()

In [23]:
data = []

for shift in range(shift_range[0], shift_range[1] + 1):
  for frame in range(frame_range[0], frame_range[1] + 1):
    if shift == 0:
      xdata = df_graph[-frame:]['oil_total']
      ydata = df_graph[-frame:]['water_total']
    else:
      xdata = df_graph[-(frame + shift):-shift]['oil_total']
      ydata = df_graph[-(frame + shift):-shift]['water_total']      

    popt, pcov = curve_fit(french_func, xdata, ydata)
    popt

    data.append([shift, frame, french_niz(*popt)])

    #print(popt[0], popt[1], np.abs(kambarov_niz(popt[0], popt[1]) - niz_total))

df_approx = pd.DataFrame(data, columns=['shift', 'frame', 'niz'])
df_approx['err'] = np.abs(df_approx['niz'] - niz_total)

In [24]:
df_approx.head(5)

,shift,frame,niz,err
0,0,10,286341.797723,277346.797723
1,0,11,197690.696842,188695.696842
2,0,12,150872.997381,141877.997381
3,0,13,120619.158981,111624.158981
4,0,14,101219.748334,92224.748334


In [25]:
fig = go.Figure()

for shift in range(shift_range[0], shift_range[1] + 1):
  df_shift_data = df_approx.query('shift == @shift')

  fig.add_trace(go.Scatter(x=df_shift_data['frame'], y=df_shift_data['err'],
                      mode='lines',
                      name='График при сдвиге: {0}'.format(shift)))

fig.update_layout( 
    xaxis=dict(
        domain=[0.2, 0.9],
        title='Размер окна',
    ),
    yaxis=dict(
        title='Ошибка предсказания',
    ),
    title={
        'text': "Зависимость ошибки предсказания НИЗ от параметров аппроксимации",
        'y':0.95,
        'x':0.50,
        'xanchor': 'center',
        'yanchor': 'top'},
    autosize=False,
    width=1400,
    height=700
)

fig.show()

In [109]:
xdata = df_graph[-25:]['oil_total']
ydata = df_graph[-25:]['water_total']

popt, pcov = curve_fit(naz_sip_func, xdata, ydata, p0 = [2, 0])
popt

print(popt)
print(xdata[-1:])
print(naz_sip_niz(np.float(ydata[-1:] - xdata[-1:]), *popt))

ydata_fit = naz_sip_func(xdata, *popt)
ydata_fit

[1.82438511e+00 9.32447872e-05]
44    7153.0
Name: oil_total, dtype: float64
7032.4095138507655


20    16027.540090
21    16500.453903
22    16854.798547
23    17243.809732
24    17556.931328
25    17838.295174
26    18108.212077
27    18394.243790
28    18720.296555
29    19029.722067
30    19345.399759
31    19685.995474
32    20065.575225
33    20409.362182
34    20787.079240
35    21153.690284
36    21549.568816
37    21948.292994
38    22386.098423
39    22843.395026
40    23298.334496
41    23702.938749
42    24101.421102
43    24460.049872
44    24860.122819
Name: oil_total, dtype: float64

In [110]:
fig = make_subplots(
    subplot_titles=[
                    'Весь период',
                    'Период аппроксимации'],
    rows=1, 
    cols=2)

fig.add_trace(go.Scatter(x=df_graph['oil_total'], y=df_graph['water_total'],
                    mode='lines+markers',
                    name='Накопленный объем нефти, 10^6'),
    row=1, col=1)

fig.add_trace(go.Scatter(x=df_graph[-35:]['oil_total'], y=df_graph[-35:]['water_total'],
                    mode='lines+markers',
                    name='Период аппроксимации'),
    row=1, col=1)

#####

fig.add_trace(go.Scatter(x=df_graph[-35:]['oil_total'], y=df_graph[-35:]['water_total'],
                    mode='lines+markers',
                    name='Годовой отбор жидкости, 10^6'),
    row=1, col=2)

fig.add_trace(go.Scatter(x=df_graph[-25:]['oil_total'], y=df_graph[-25:]['water_total'],
                    mode='lines+markers',
                    name='Участок аппроксимации'),
    row=1, col=2)

fig.add_trace(go.Scatter(x=df_graph[-25:]['oil_total'], y=naz_sip_func(xdata, *popt),
                    mode='lines',
                    name='Функция аппроксимации (Назаров-Сипачев)'),
    row=1, col=2)


fig['layout']['xaxis1']['title']='Накопленный объем нефти, 10^-6'
fig['layout']['xaxis1']['tickangle']=-45
fig['layout']['xaxis1']['tickmode']='array'
fig['layout']['yaxis1']['title']='Накопленный объем жидкости, 10^6'

fig['layout']['xaxis2']['title']='Накопленный объем нефти, 10^-6'
fig['layout']['xaxis2']['tickangle']=-45



fig.update_layout( 
    title={
        'text': 'Зависимость накопленного объема жидкости от добычи нефти',
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'},
    autosize=False,
    width=1400,
    height=700)


fig.show()

In [71]:
ydata_fit[-1:]

34    21839.75787
Name: oil_total, dtype: float64

In [72]:
xdata[-1:]

34    6606.0
Name: oil_total, dtype: float64

In [74]:
naz_sip_niz(np.float(ydata[-1:] - xdata[-1:]), *popt)

6570.450395472272

In [97]:
vvk = np.float(ydata[-1:] - xdata[-1:])
vvk = 23993
vvk

23993

In [77]:
popt[1]

0.00011503603691330729

In [82]:
vvk / (popt[0] - 1 + popt[1] * vvk)

6760.813665876672

In [94]:
frame_range = [3, 20]
shift_range = [0, 12]

In [98]:
data = []

for shift in range(shift_range[0], shift_range[1] + 1):
  for frame in range(frame_range[0], frame_range[1] + 1):
    if shift == 0:
      xdata = df_graph[-frame:]['oil_total']
      ydata = df_graph[-frame:]['water_total']
    else:
      xdata = df_graph[-(frame + shift):-shift]['oil_total']
      ydata = df_graph[-(frame + shift):-shift]['water_total']      

    popt, pcov = curve_fit(naz_sip_func, xdata, ydata, p0 = [2, 0])
    popt

    #print(naz_sip_niz(np.float(ydata[-1:] - xdata[-1:]), *popt))
    
    #data.append([shift, frame, naz_sip_niz(np.float(ydata[-1:] - xdata[-1:]), *popt)])
    data.append([shift, frame, naz_sip_niz(vvk, *popt)])

    
    #if (shift == 1 and frame == 6):
      #print(naz_sip_niz(np.float(ydata[-1:] - xdata[-1:]), *popt))

    #print(popt[0], popt[1], np.abs(kambarov_niz(popt[0], popt[1]) - niz_total))

df_approx = pd.DataFrame(data, columns=['shift', 'frame', 'niz'])
df_approx['err'] = np.abs(df_approx['niz'] - niz_total)

In [107]:
df_approx.head(5)

,shift,frame,niz,err
0,0,3,8738.655338,256.344662
1,0,4,8877.380673,117.619327
2,0,5,9032.180154,37.180154
3,0,6,9211.621147,216.621147
4,0,7,9388.930974,393.930974


In [100]:
print_approx_graph(df_approx)

In [106]:
plot_approx_heatmap(df_approx.pivot(index='shift', columns='frame', values='err'))

In [101]:
xdata = df_graph[-11:-3]['oil_total']
ydata = df_graph[-11:-3]['water_total']

popt, pcov = curve_fit(naz_sip_func, xdata, ydata, p0 = [2, 0])
popt
naz_sip_niz(vvk, *popt)

9009.48401815127

In [108]:
fig = make_subplots(
    subplot_titles=[
                    'Весь период',
                    'Период аппроксимации'],
    rows=1, 
    cols=2)

fig.add_trace(go.Scatter(x=df_graph['oil_total'], y=df_graph['water_total'],
                    mode='lines+markers',
                    name='Накопленный объем нефти, 10^6'),
    row=1, col=1)

fig.add_trace(go.Scatter(x=df_graph[-35:]['oil_total'], y=df_graph[-35:]['water_total'],
                    mode='lines+markers',
                    name='Период аппроксимации'),
    row=1, col=1)

#####

fig.add_trace(go.Scatter(x=df_graph[-35:]['oil_total'], y=df_graph[-35:]['water_total'],
                    mode='lines+markers',
                    name='Годовой отбор жидкости, 10^6'),
    row=1, col=2)

fig.add_trace(go.Scatter(x=df_graph[-11:-3]['oil_total'], y=df_graph[-11:-3]['water_total'],
                    mode='lines+markers',
                    name='Участок аппроксимации'),
    row=1, col=2)

fig.add_trace(go.Scatter(x=df_graph[-11:-3]['oil_total'], y=naz_sip_func(xdata, *popt),
                    mode='lines',
                    name='Функция аппроксимации (Камбарова)'),
    row=1, col=2)

fig.add_annotation(x=5500, y=23000, text="НИЗ={0}".format(int(naz_sip_niz(vvk, *popt))), showarrow=False, yshift=10, font=anno_font,
    row=1, col=2)

fig['layout']['xaxis1']['title']='Накопленный объем нефти, 10^-6'
fig['layout']['xaxis1']['tickangle']=-45
fig['layout']['xaxis1']['tickmode']='array'
fig['layout']['yaxis1']['title']='Накопленный объем жидкости, 10^6'

fig['layout']['xaxis2']['title']='Накопленный объем нефти, 10^-6'
fig['layout']['xaxis2']['tickangle']=-45



fig.update_layout( 
    title={
        'text': 'Зависимость накопленного объема жидкости от добычи нефти',
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'},
    autosize=False,
    width=1400,
    height=700)


fig.show()

In [38]:
xdata = df_graph[-25:-5]['water_total']
ydata = df_graph[-25:-5]['oil_total']

popt, pcov = curve_fit(sazonov_func, xdata, ydata)
popt

ydata_fit = func_sazonov(xdata, *popt)

NameError: ignored

In [ ]:
fig = make_subplots(
    subplot_titles=[
                    'Весь период',
                    'Период аппроксимации'],
    rows=1, 
    cols=2)

fig.add_trace(go.Scatter(x=df_graph['water_total_ln'], y=df_graph['oil_total'],
                    mode='lines+markers',
                    name='Накопленный объем нефти, 10^6'),
    row=1, col=1)

fig.add_trace(go.Scatter(x=df_graph[-35:]['water_total_ln'], y=df_graph[-35:]['oil_total'],
                    mode='lines+markers',
                    name='Период аппроксимации'),
    row=1, col=1)

#####

fig.add_trace(go.Scatter(x=df_graph[-35:]['water_total_ln'], y=df_graph[-35:]['oil_total'],
                    mode='lines+markers',
                    name='Годовой отбор жидкости, 10^6'),
    row=1, col=2)

fig.add_trace(go.Scatter(x=df_graph[-25:-5]['water_total_ln'], y=df_graph[-25:]['oil_total'],
                    mode='lines+markers',
                    name='Участок аппроксимации'),
    row=1, col=2)

fig.add_trace(go.Scatter(x=df_graph[-25:-5]['water_total_ln'], y=ydata_fit,
                    mode='lines',
                    name='Функция аппроксимации (Сазонова)'),
    row=1, col=2)


fig['layout']['xaxis1']['title']='ln(накопленный объем жидкости)'
fig['layout']['xaxis1']['tickangle']=-45
fig['layout']['xaxis1']['tickmode']='array'
#fig['layout']['xaxis1']['tickvals']=df_data['year']
fig['layout']['yaxis1']['title']='Накопленный объем нефти, 10^6'

fig['layout']['xaxis2']['title']='ln(накопленный объем жидкости)'
fig['layout']['xaxis2']['tickangle']=-45
#fig['layout']['yaxis2']['title']='Обводненность, д.ед.'



fig.update_layout( 
    title={
        'text': 'Зависимость накопленной добычи нефти от объема жидкости',
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'},
    autosize=False,
    width=1400,
    height=700)


fig.show()

In [ ]:
frame_range = [10, 25]
shift_range = [0, 15]

In [ ]:
for shift in range(shift_range[0], shift_range[1] + 1):
  for frame in range(frame_range[0], frame_range[1] + 1):
    if shift == 0:
      xdata = df_graph[-frame:]['water_total']
      ydata = df_graph[-frame:]['oil_total']
    else:
      xdata = df_graph[-(frame + shift):-shift]['water_total']
      ydata = df_graph[-(frame + shift):-shift]['oil_total']      

    #print(shift, frame)
    #popt, pcov = curve_fit(func_kambarov, xdata, ydata)
    popt, pcov = curve_fit(func_sazonov, xdata, ydata)
    popt

    #print(popt[0], popt[1], kambarov_niz(popt[0], np.abs(popt[1])))
    print(popt[0], popt[1], sazonov_niz(popt[0], np.abs(popt[1])))


    #ydata_fit = func_kambarov(xdata, *popt)


In [ ]:
df_graph[-27:-2]['oil_total']

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_graph['water_total_deg_1'], y=df_graph['oil_total'],
                    mode='lines+markers',
                    name='Накопленный объем нефти, 10^6'))

fig.add_trace(go.Scatter(x=df_graph[-35:]['water_total_deg_1'], y=df_graph[-35:]['oil_total'],
                    mode='lines',
                    name='Накопленный объем нефти <br>(предполагаемый период масштабирования)'))

fig.update_layout( 
    xaxis=dict(
        domain=[0.2, 0.9],
        title='1 / Накопленный объем жидкости',
    ),
    yaxis=dict(
        title='Накопленный объем нефти',
    ),
    title={
        'text': "Зависимость накопленной добычи нефти от объема жидкости",
        'y':0.95,
        'x':0.50,
        'xanchor': 'center',
        'yanchor': 'top'},
    autosize=False,
    width=1400,
    height=900
)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_graph[-35:]['water_total_deg_1'], y=df_graph[-35:]['oil_total'],
                    mode='lines+markers',
                    name='Годовой отбор жидкости, 10^6'))

fig.add_trace(go.Scatter(x=df_graph[-25:]['water_total_deg_1'], y=df_graph[-25:]['oil_total'],
                    mode='lines+markers',
                    name='Годовой отбор жидкости, 10^6'))

fig.add_trace(go.Scatter(x=xdata, y=ydata_fit,
                    mode='lines',
                    name='Годовой отбор жидкости, 10^6'))

fig.update_layout( 
    xaxis=dict(
        domain=[0.2, 0.9],
        title='Накопленный объем жидкости в степени -1',
    ),
    title={
        'text': "Зависимость накопленной добычи нефти от объема жидкости",
        'y':0.95,
        'x':0.55,
        'xanchor': 'center',
        'yanchor': 'top'},
    autosize=False,
    width=1400,
    height=900
)

fig.show()

In [ ]:
popt, pcov = curve_fit(func_kambarov, xdata, ydata)
popt

plt.plot(xdata, func_kambarov(xdata, *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f' % tuple(popt))

In [ ]:
popt
b = popt[1]

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def func(x, a, b, c):
    return a * np.exp(-b * x) + c

xdata = np.linspace(0, 4, 50)
y = func(xdata, 2.5, 1.3, 0.5)
np.random.seed(1729)
y_noise = 0.2 * np.random.normal(size=xdata.size)
ydata = y + y_noise
plt.plot(xdata, ydata, 'b-', label='data')

In [ ]:
def func2(x, a, b):
    return a - b * x

In [ ]:
popt, pcov = curve_fit(func2, xdata, ydata)
popt

plt.plot(xdata, func2(xdata, *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f' % tuple(popt))

In [ ]:
popt, pcov = curve_fit(func, xdata, ydata, bounds=(0, [3., 1., 0.5]))
popt

plt.plot(xdata, func(xdata, *popt), 'g--',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()